### Pandas equivalent example to work with Document Clustering for People Wiki dataframe

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
people = pd.read_csv(f"D:/people_wiki.csv")
#people = pd.read_csv(f"D:/SYED/people_wiki.csv")
people.head(3)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...


In [3]:
obama = people[people['name'] == 'Barack Obama']
obama

,URI,name,text
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(obama['text'])
matrix

<1x270 sparse matrix of type '<class 'numpy.int64'>'
	with 270 stored elements in Compressed Sparse Row format>

In [ ]:
counts = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
print(counts)

In [11]:
counts.T.sort_values(by=0, ascending=False).head(10)

,0
the,40
in,30
and,21
of,18
to,14
his,11
obama,9
act,8
he,7
as,6


In [7]:
counts['obama']

0    9
Name: obama, dtype: int64

In [8]:
people.head(3)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...


In [ ]:
pep1 = people[10:100]
pep1

In [17]:
pep1.rename(columns = {'text':'text_data'}, inplace = True)
pep1.head(3)

C:\Users\SYED\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,URI,name,text_data
10,<http://dbpedia.org/resource/Sophie_Crumb>,Sophie Crumb,sophia violet sophie crumb born september 27 1...
11,<http://dbpedia.org/resource/Jenn_Ashworth>,Jenn Ashworth,jenn ashworth is an english writer she was bor...
12,<http://dbpedia.org/resource/Jonathan_Hoefler>,Jonathan Hoefler,jonathan hoefler born august 22 1970 is an ame...


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

def final_pop(initial_pop,growth_rate):
    final = initial_pop*math.e**(growth_rate*35)
    return(final)

def make_counts(text_data):
    cv = CountVectorizer()
    cv_fit = vectorizer.fit_transform(text_data)
    word_list = cv.get_feature_names_out()
    count_list = np.asarray(cv_fit.sum(axis=0))[0]
    final_dict = dict(zip(word_list, count_list))
    return final_dict

In [29]:
pep1['word_count'] = pep1['text_data'].apply(lambda x: make_counts([x]))

NotFittedError: Vocabulary not fitted or provided